In [14]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')


def naive_nodewise_regression(Y_star, lambda_grid=None):
    """
    Implements Naive Nodewise Regression (Section 5.1.2).
    Uses GIC (Generalized Information Criterion) as in the paper.
    
    Parameters:
    -----------
    Y_star : np.ndarray, shape (n, p)
        Demeaned returns matrix (time x assets)
    lambda_grid : list or None
        Grid of lambda values to try. If None, creates default grid.
    
    Returns:
    --------
    Theta_hat : np.ndarray, shape (p, p)
        Estimated precision matrix
    """
    n, p = Y_star.shape

    # Initialize matrices
    Theta_hat = np.zeros((p, p))
    tau_squared = np.zeros(p)
    
    # Create lambda grid if not provided
    if lambda_grid is None:
        lambda_grid = np.logspace(-3, 1, 50)
    
    # For each asset j
    for j in range(p):
        # Step 1: Get y_j (target) and Y_{-j} (predictors)
        y_j = Y_star[:, j]
        Y_minus_j = np.delete(Y_star, j, axis=1)
        
        # Step 2-3: Estimate gamma_j using Lasso with GIC
        best_gic = np.inf
        best_lambda = lambda_grid[0]
        best_gamma = None
        best_ssr = None
        
        for lam in lambda_grid:
            lasso = Lasso(alpha=2*lam, fit_intercept=False, max_iter=10000)
            lasso.fit(Y_minus_j, y_j)
            gamma_j = lasso.coef_
            
            # Compute SSR and number of non-zero coefficients
            residuals = y_j - Y_minus_j @ gamma_j
            ssr = np.sum(residuals ** 2)
            sigma_sq_lambda = ssr / n
            q_lambda = np.sum(np.abs(gamma_j) > 1e-8)
            
            # Compute GIC
            if sigma_sq_lambda > 1e-10:
                gic = np.log(sigma_sq_lambda) + q_lambda * (np.log(p) / n) * np.log(np.log(n))
            else:
                gic = np.inf
            
            if gic < best_gic:
                best_gic = gic
                best_lambda = lam
                best_gamma = gamma_j.copy()
                best_ssr = ssr
        
        gamma_j_star = best_gamma
        tau_squared[j] = best_ssr / n + best_lambda * np.sum(np.abs(gamma_j_star))

        # Step 5: Form the j-th row of Theta_hat
        Theta_hat[j, j] = 1 / tau_squared[j]
        off_diag = -gamma_j_star / tau_squared[j]
        Theta_hat[j, :j] = off_diag[:j]
        Theta_hat[j, j+1:] = off_diag[j:]
    
    # Step 6: Symmetrize
    Theta_hat_sym = (Theta_hat + Theta_hat.T) / 2
    
    return Theta_hat_sym


def gmv_weights(Theta_hat):
    """
    Compute Global Minimum Variance (GMV) portfolio weights (Section 6.1).
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # w* = (Θ 1_p) / (1_p' Θ 1_p)
    numerator = Theta_hat @ ones_p
    denominator = ones_p @ Theta_hat @ ones_p
    
    if np.abs(denominator) < 1e-10:
        # Fallback to equal weights if precision matrix is near-singular
        return ones_p / p
    
    w_star = numerator / denominator
    
    return w_star


def mv_weights(Theta_hat, mu, target_return=0.01):
    """
    Compute Mean-Variance portfolio weights with target return.
    
    Solves the constrained optimization:
    min w' Sigma w  subject to  w' mu = target_return  and  w' 1 = 1
    
    Solution uses Lagrange multipliers with two constraints.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected returns
    target_return : float
        Target portfolio return (default: 0.01 = 1% monthly)
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute key quantities
    A = ones_p @ Theta_hat @ ones_p  # 1' Theta 1
    B = ones_p @ Theta_hat @ mu       # 1' Theta mu  
    C = mu @ Theta_hat @ mu           # mu' Theta mu
    D = A * C - B * B                  # Determinant
    
    # Check for singularity
    if np.abs(D) < 1e-10:
        # System is singular, use GMV instead
        if np.abs(A) > 1e-10:
            w_star = (Theta_hat @ ones_p) / A
            return w_star
        else:
            return ones_p / p
    
    # Compute Lagrange multipliers
    lambda1 = (C - B * target_return) / D
    lambda2 = (A * target_return - B) / D
    
    # Compute weights: w = lambda1 * Theta^{-1} 1 + lambda2 * Theta^{-1} mu
    w_star = lambda1 * (Theta_hat @ ones_p) + lambda2 * (Theta_hat @ mu)
    
    return w_star


def msr_weights(Theta_hat, mu):
    """
    Compute Maximum Sharpe Ratio portfolio weights.
    
    The maximum Sharpe ratio portfolio solves:
    max (w' mu) / sqrt(w' Sigma w)
    
    Solution (when mu represents excess returns):
    w ∝ Sigma^{-1} mu = Theta mu
    
    Then normalize so that sum(w) = 1.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected excess returns
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights (sum to 1)
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute unnormalized weights: w ∝ Theta mu
    w_unnorm = Theta_hat @ mu
    
    # Normalize to sum to 1
    weight_sum = np.sum(w_unnorm)
    
    if np.abs(weight_sum) < 1e-10:
        print('WARNING: Weight sum near zero, returning equal weights')
        return ones_p / p
    
    w_star = w_unnorm / weight_sum
    
    return w_star


def load_recommendation_changes(rec_changes_path):
    """
    Load recommendation changes from CSV file.
    
    Parameters:
    -----------
    rec_changes_path : str
        Path to monthly_mean_recommendations_decay.csv file
    
    Returns:
    --------
    rec_changes_df : pd.DataFrame
        DataFrame with columns: permno, date, ticker, weighted_mean_recommendation, 
        recommendation_change, num_recommendations
    """
    try:
        rec_changes_df = pd.read_csv(rec_changes_path)
        rec_changes_df['date'] = pd.to_datetime(rec_changes_df['date'])
        rec_changes_df['permno'] = rec_changes_df['permno'].astype(int)
        return rec_changes_df
    except FileNotFoundError as e:
        print(f"  ⚠ Warning: Could not load recommendation changes: {e}")
        return pd.DataFrame(columns=['permno', 'date', 'ticker', 'weighted_mean_recommendation', 
                                    'recommendation_change', 'num_recommendations'])


def get_signal_permnos_for_date(rec_changes_df, date, buy_threshold=-0.5, sell_threshold=0.5):
    """
    Get sets of permnos with buy/sell signals based on recommendation changes.
    
    Note: Negative change = upgrade (moving toward Strong Buy) = BUY signal
          Positive change = downgrade (moving toward Sell) = SELL signal
    
    Parameters:
    -----------
    rec_changes_df : pd.DataFrame
        Recommendation changes dataframe
    date : pd.Timestamp
        Date to get signals for
    buy_threshold : float
        Threshold for buy signals (default: -0.5)
    sell_threshold : float
        Threshold for sell signals (default: 0.5)
    
    Returns:
    --------
    buy_permnos : set
        Set of permnos with buy signals
    sell_permnos : set
        Set of permnos with sell signals
    """
    date_changes = rec_changes_df[rec_changes_df['date'] == date]
    
    # Buy signals: negative changes (recommendations getting better)
    buys = date_changes[date_changes['recommendation_change'] <= buy_threshold]
    buy_permnos = set(buys['permno'].values)
    
    # Sells signals: positive changes (recommendations getting worse)
    sells = date_changes[date_changes['recommendation_change'] >= sell_threshold]
    sell_permnos = set(sells['permno'].values)
    
    return buy_permnos, sell_permnos


def backtest_nodewise_gmv_analyst(df, 
                                     test_start_date='2020-01-31', 
                                     test_end_date='2024-11-30',
                                     lookback_window=180,
                                     transaction_cost=0.005,
                                     rec_changes_path=None,
                                     buy_threshold=-0.5,
                                     sell_threshold=0.5,
                                     target_return=0.01,
                                     verbose=True):
    """
    Backtest all three portfolio strategies (GMV, MV, MSR) simultaneously.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: permno, datadate, ret_fwd_1
    test_start_date : str
        First date for out-of-sample returns (format: 'YYYY-MM-DD')
    test_end_date : str
        Last date for out-of-sample returns (format: 'YYYY-MM-DD')
    lookback_window : int
        Number of months in rolling training window (default: 180)
    transaction_cost : float
        Proportional transaction cost (default: 0.005 = 50 bps)
    rec_changes_path : str
        Path to monthly_mean_recommendations_decay.csv file (required)
    buy_threshold : float
        Threshold for buy signals (default: -0.5)
    sell_threshold : float
        Threshold for sell signals (default: 0.5)
    target_return : float
        Target return for MV portfolio (default: 0.01 = 1% monthly)
    verbose : bool
        If True, prints detailed log at each time step.
    
    Returns:
    --------
    results : dict
        Dictionary with keys 'gmv', 'mv', 'msr', each containing:
        - results_df : pd.DataFrame with columns: date, portfolio_return, cumulative_return
        - metrics : dict with overall performance metrics
    """
    
    # --- 1. Setup ---
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])
    
    # Load recommendation changes (required)
    if rec_changes_path is None:
        raise ValueError("rec_changes_path is required for this strategy")
    
    rec_changes_df = load_recommendation_changes(rec_changes_path)
    if len(rec_changes_df) == 0:
        raise ValueError("No recommendation changes loaded")
    
    if verbose:
        print(f"Loaded recommendation changes: {len(rec_changes_df)} records")
        print(f"Computing ALL portfolio types: GMV, MV (target={target_return}), MSR")
        print(f"Strategy: BUY threshold <= {buy_threshold}, SELL threshold >= {sell_threshold}")
    
    # Get unique dates
    all_dates = sorted(df['datadate'].unique())
    
    # Convert test dates to datetime
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    # Find date indices
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    if test_start_idx < lookback_window:
        raise ValueError(f"Not enough data for lookback. Test start date {test_start_date} "
                         f"requires data back to {all_dates[test_start_idx - lookback_window]}, "
                         f"but only {test_start_idx} periods are available.")
    
    # Storage for results - one set per portfolio type
    results_storage = {
        'gmv': {'returns': [], 'dates': [], 'weights': [], 'turnover': [], 'gross_returns': [],
                'prev_weights': {}, 'prev_oos_returns': {}, 'prev_gross_return': 0.0},
        'mv': {'returns': [], 'dates': [], 'weights': [], 'turnover': [], 'gross_returns': [],
               'prev_weights': {}, 'prev_oos_returns': {}, 'prev_gross_return': 0.0},
        'msr': {'returns': [], 'dates': [], 'weights': [], 'turnover': [], 'gross_returns': [],
                'prev_weights': {}, 'prev_oos_returns': {}, 'prev_gross_return': 0.0}
    }
    
    # --- 2. Rolling Window Backtest ---
    if verbose:
        print("="*60)
        print(f"STARTING BACKTEST FOR ALL PORTFOLIOS (GMV, MV, MSR)")
        print("="*60)
        
    for t in range(test_start_idx, test_end_idx + 1):
        current_date = all_dates[t]
        
        # Get buy and sell signals for current date
        buy_permnos, sell_permnos = get_signal_permnos_for_date(
            rec_changes_df, current_date, buy_threshold, sell_threshold
        )
        
        # Combine all permnos with signals
        allowed_permnos = buy_permnos | sell_permnos
        
        # Define the lookback window
        window_start_date = all_dates[t - lookback_window]
        window_end_date = all_dates[t - 1]
        
        # Get training data for this window, filtered by allowed permnos
        train_data = df[(df['datadate'] >= window_start_date) & 
                        (df['datadate'] <= window_end_date) &
                        (df['permno'].isin(allowed_permnos))]
        
        # Pivot to get returns matrix (time x assets)
        returns_pivot = train_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        
        # Reindex to ensure all dates are present
        window_dates = all_dates[t - lookback_window : t]
        returns_pivot = returns_pivot.reindex(index=window_dates)
        
        # Filter assets with any NaNs in this window
        nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
        filtered_pivot = returns_pivot.drop(columns=nan_assets)
        
        current_assets = filtered_pivot.columns.tolist()
        Y = filtered_pivot.values
        n_train, p_current = Y.shape

        if verbose:
            print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                  f"Date: {current_date.strftime('%Y-%m-%d')}")
            print(f"  Window: {window_start_date.strftime('%Y-%m-%d')} to "
                  f"{window_end_date.strftime('%Y-%m-%d')}")
            print(f"  Buys: {len(buy_permnos)} | Sells: {len(sell_permnos)} | "
                  f"Assets w/ data: {p_current}")

        # Check for valid data
        if n_train < lookback_window or p_current < 2:
            if verbose:
                print(f"  ⚠ Insufficient data (n={n_train}, p={p_current}), using prev weights")
            # Use previous weights for all portfolios
            weights_dict = {
                'gmv': results_storage['gmv']['prev_weights'].copy(),
                'mv': results_storage['mv']['prev_weights'].copy(),
                'msr': results_storage['msr']['prev_weights'].copy()
            }
        else:
            try:
                # Demean the returns
                Y_bar = Y.mean(axis=0)
                Y_star = Y - Y_bar
                
                if verbose:
                    print(f"  Running Nodewise Regression...")
                Theta_hat = naive_nodewise_regression(Y_star)
                
                # Compute all three portfolio weights
                if verbose:
                    print(f"  Computing portfolio weights: GMV, MV, MSR...")
                
                w_gmv = gmv_weights(Theta_hat)
                w_mv = mv_weights(Theta_hat, Y_bar, target_return=target_return)
                w_msr = msr_weights(Theta_hat, Y_bar)
                
                # Create weights dictionaries
                weights_dict = {
                    'gmv': {asset: w_gmv[i] for i, asset in enumerate(current_assets)},
                    'mv': {asset: w_mv[i] for i, asset in enumerate(current_assets)},
                    'msr': {asset: w_msr[i] for i, asset in enumerate(current_assets)}
                }
                
            except Exception as e:
                if verbose:
                    print(f"  ✗ Error: {e}")
                    print(f"  Using previous weights")
                weights_dict = {
                    'gmv': results_storage['gmv']['prev_weights'].copy(),
                    'mv': results_storage['mv']['prev_weights'].copy(),
                    'msr': results_storage['msr']['prev_weights'].copy()
                }

        # Get out-of-sample returns for current month
        oos_data = df[df['datadate'] == current_date]
        oos_returns_series = oos_data.set_index('permno')['ret_fwd_1']
        oos_returns_series = oos_returns_series.dropna()
        oos_returns_dict = oos_returns_series.to_dict()
        
        # Process each portfolio type
        for port_type in ['gmv', 'mv', 'msr']:
            new_weights_dict = weights_dict[port_type]
            prev_weights_dict = results_storage[port_type]['prev_weights']
            prev_oos_returns_dict = results_storage[port_type]['prev_oos_returns']
            prev_gross_return = results_storage[port_type]['prev_gross_return']
            
            # Normalize weights to sum to 1
            weight_sum = sum(new_weights_dict.values())
            if weight_sum > 1e-10:
                new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
            else:
                if verbose and port_type == 'gmv':  # Only print once
                    print("  ⚠ Zero weight sum, using previous weights")
                new_weights_dict = prev_weights_dict.copy()
            
            # Find common assets between weights and returns
            common_assets = set(new_weights_dict.keys()) & set(oos_returns_dict.keys())
            
            if len(common_assets) == 0:
                if verbose and port_type == 'gmv':  # Only print once
                    print(f"  ⚠ No common assets with valid returns, skipping period")
                continue
            
            # Filter to common assets and renormalize
            common_weights = {a: new_weights_dict[a] for a in common_assets}
            common_weight_sum = sum(common_weights.values())
            if common_weight_sum > 1e-10:
                common_weights = {k: v/common_weight_sum for k, v in common_weights.items()}
            else:
                if verbose and port_type == 'gmv':  # Only print once
                    print(f"  ⚠ Zero weight sum after filtering, skipping period")
                continue
            
            # Compute gross portfolio return
            gross_return = sum(common_weights[a] * oos_returns_dict[a] for a in common_assets)
            
            # Sanity check
            if np.isnan(gross_return) or np.isinf(gross_return):
                if verbose and port_type == 'gmv':  # Only print once
                    print(f"  ⚠ Invalid gross return: {gross_return}, skipping period")
                continue
            
            # Calculate transaction costs
            if len(prev_weights_dict) > 0:
                # Adjust ALL previous weights for returns
                adjusted_prev = {}
                for asset, prev_w in prev_weights_dict.items():
                    if asset in prev_oos_returns_dict:
                        prev_r = prev_oos_returns_dict[asset]
                        if abs(1 + prev_gross_return) > 1e-6:
                            adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
                        else:
                            adjusted_prev[asset] = 0.0
                    else:
                        if abs(1 + prev_gross_return) > 1e-6:
                            adjusted_prev[asset] = prev_w / (1 + prev_gross_return)
                        else:
                            adjusted_prev[asset] = 0.0
                
                # Calculate turnover
                all_assets = set(adjusted_prev.keys()) | set(common_weights.keys())
                turnover = 0.0
                for asset in all_assets:
                    old_w = adjusted_prev.get(asset, 0.0)
                    new_w = common_weights.get(asset, 0.0)
                    turnover += abs(new_w - old_w)
                
                tc = transaction_cost * (1 + gross_return) * turnover
            else:
                # First period
                turnover = sum(abs(w) for w in common_weights.values())
                tc = transaction_cost * (1 + gross_return) * turnover
            
            # Net return
            net_return = gross_return - tc
            
            # Store results for this portfolio type
            results_storage[port_type]['returns'].append(net_return)
            results_storage[port_type]['dates'].append(current_date)
            results_storage[port_type]['weights'].append(common_weights.copy())
            results_storage[port_type]['turnover'].append(turnover)
            results_storage[port_type]['gross_returns'].append(gross_return)
            
            # Update previous values
            results_storage[port_type]['prev_weights'] = common_weights.copy()
            results_storage[port_type]['prev_oos_returns'] = {a: oos_returns_dict[a] for a in common_assets}
            results_storage[port_type]['prev_gross_return'] = gross_return
        
        if verbose:
            # Print results for all three portfolios
            print(f"  GMV - Gross: {results_storage['gmv']['gross_returns'][-1] if results_storage['gmv']['gross_returns'] else 'N/A':>8.5f} | "
                  f"Net: {results_storage['gmv']['returns'][-1] if results_storage['gmv']['returns'] else 'N/A':>8.5f}")
            print(f"  MV  - Gross: {results_storage['mv']['gross_returns'][-1] if results_storage['mv']['gross_returns'] else 'N/A':>8.5f} | "
                  f"Net: {results_storage['mv']['returns'][-1] if results_storage['mv']['returns'] else 'N/A':>8.5f}")
            print(f"  MSR - Gross: {results_storage['msr']['gross_returns'][-1] if results_storage['msr']['gross_returns'] else 'N/A':>8.5f} | "
                  f"Net: {results_storage['msr']['returns'][-1] if results_storage['msr']['returns'] else 'N/A':>8.5f}")

    if verbose:
        print("\n" + "="*60)
        print("BACKTEST COMPLETE - ALL PORTFOLIOS")
        print("="*60)
    
    # --- 4. Compile Results for All Portfolios ---
    results = {}
    
    for port_type in ['gmv', 'mv', 'msr']:
        portfolio_returns = results_storage[port_type]['returns']
        portfolio_dates = results_storage[port_type]['dates']
        portfolio_gross_returns = results_storage[port_type]['gross_returns']
        portfolio_weights_list = results_storage[port_type]['weights']
        portfolio_turnover_list = results_storage[port_type]['turnover']
        
        results_df = pd.DataFrame({
            'date': portfolio_dates,
            'portfolio_return': portfolio_returns,
            'portfolio_gross_return': portfolio_gross_returns,
            'portfolio_weights': portfolio_weights_list,
            'portfolio_turnover': portfolio_turnover_list
        })
        
        if len(results_df) > 0:
            results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
        
        # Compute overall metrics
        if len(portfolio_returns) > 0:
            mean_return = np.mean(portfolio_returns)
            variance = np.var(portfolio_returns, ddof=1)
            sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
            
            # Annualized metrics (monthly data)
            annual_return = mean_return * 12
            annual_volatility = np.sqrt(variance * 12)
            annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
            
            metrics = {
                'mean_return': mean_return,
                'variance': variance,
                'sharpe_ratio': sharpe_ratio,
                'annual_return': annual_return,
                'annual_volatility': annual_volatility,
                'annual_sharpe_ratio': annual_sharpe,
                'total_return': results_df['cumulative_return'].iloc[-1],
                'avg_turnover': np.mean(portfolio_turnover_list),
                'n_periods': len(portfolio_returns),
                'portfolio_type': port_type
            }
        else:
            metrics = {
                'mean_return': 0,
                'variance': 0,
                'sharpe_ratio': 0,
                'annual_return': 0,
                'annual_volatility': 0,
                'annual_sharpe_ratio': 0,
                'total_return': 0,
                'avg_turnover': 0,
                'n_periods': 0,
                'portfolio_type': port_type
            }
        
        results[port_type] = {
            'results_df': results_df,
            'metrics': metrics
        }
    
    return results

In [11]:
df = pd.read_csv('../green cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = df.groupby('permno')['ret_excess'].shift(-1)

In [15]:
results = backtest_nodewise_gmv_analyst(
    df=df,
    test_start_date='2020-01-31',
    test_end_date='2024-04-30',
    lookback_window=180,
    transaction_cost=0.001,
    rec_changes_path='monthly_mean_recommendations_decay.csv',  # Required!
    verbose=True
)

# Access individual portfolio results
gmv_df = results['gmv']['results_df']
gmv_metrics = results['gmv']['metrics']

mv_df = results['mv']['results_df']
mv_metrics = results['mv']['metrics']

msr_df = results['msr']['results_df']
msr_metrics = results['msr']['metrics']

# Compare Sharpe ratios
print(f"GMV Sharpe: {gmv_metrics['annual_sharpe_ratio']:.4f}")
print(f"MV Sharpe:  {mv_metrics['annual_sharpe_ratio']:.4f}")
print(f"MSR Sharpe: {msr_metrics['annual_sharpe_ratio']:.4f}")

Loaded recommendation changes: 16960 records
Computing ALL portfolio types: GMV, MV (target=0.01), MSR
Strategy: BUY threshold <= -0.5, SELL threshold >= 0.5
STARTING BACKTEST FOR ALL PORTFOLIOS (GMV, MV, MSR)

[1/52] Date: 2020-01-31
  Window: 2005-01-31 to 2019-12-31
  Buys: 52 | Sells: 19 | Assets w/ data: 41
  Running Nodewise Regression...
  Computing portfolio weights: GMV, MV, MSR...
  GMV - Gross: -0.09592 | Net: -0.09684
  MV  - Gross: -0.09498 | Net: -0.09590
  MSR - Gross: -0.08561 | Net: -0.08658

[2/52] Date: 2020-02-29
  Window: 2005-02-28 to 2020-01-31
  Buys: 35 | Sells: 38 | Assets w/ data: 38
  Running Nodewise Regression...
  Computing portfolio weights: GMV, MV, MSR...
  GMV - Gross: -0.13832 | Net: -0.13948
  MV  - Gross: -0.13582 | Net: -0.13699
  MSR - Gross: -0.11491 | Net: -0.11616

[3/52] Date: 2020-03-31
  Window: 2005-03-31 to 2020-02-29
  Buys: 132 | Sells: 42 | Assets w/ data: 87
  Running Nodewise Regression...
  Computing portfolio weights: GMV, MV, MSR.